In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
#from urllib.parse import urlparse
import pandas as pd
import numpy as np
import os

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [5]:
from collections import namedtuple

def get_ext_links_df_from_url(url=None):
    """phrase qui resume
    
    Parameters
    ----------
    url: string, default=None
        Base URL to scrap links

    Returns
    -------
    df: Pandas.DataFrame
        Dataframe containing the links
    """
    UrlDescription = namedtuple('UrlDescription', ['href', 'text'])
    
    raw_html = simple_get(url)
    html = BeautifulSoup(raw_html, 'html.parser')
    df = pd.DataFrame(columns=['text','href'])
    
    # Get all (http links, description) from 
    # current url and store in dataframe
    urls_descriptions = []
    for a in html.find_all('a', href=True):
        if (a['href'].startswith("http")):
            urls_descriptions.append(UrlDescription(text=a.text, href=a['href']))
            #df=df.append([{'text':a.text, 'href':a['href']}],ignore_index=True)
    
    df = pd.DataFrame(urls_descriptions)
    return df

In [12]:
url="https://fr.wikipedia.org/wiki/Presse_en_France"
df=get_ext_links_df_from_url(url)
df

,href,text
0,http://www.acpm.fr/Support/aujourd-hui-en-france,« Aujourd'hui en France »
1,http://www.acpm.fr/Support/le-parisien,« Le Parisien »
2,http://www.audipresse.fr/node.php?id=1220&elem...,« ONE 2011 »
3,http://www.audipresse.fr/media/document/one201...,« ONE 2012 »
4,http://www.pqr.fr/wp-content/uploads/2011/03/E...,« EPIQ 2010 »
5,http://www.acpm.fr/var/ojd/storage/files/books...,« Book - Presse payante 2011 »
6,http://www.acpm.fr/var/ojd/storage/files/books...,« Book - Presse payante 2012 »
7,http://www.acpm.fr/var/ojd/storage/files/books...,« Book - Presse payante 2013 »
8,http://www.acpm.fr/var/ojd/storage/files/books...,« Book - Presse payante 2014 »
9,http://www.acpm.fr/var/ojd/storage/files/books...,« Book - Presse payante 2015 »


In [6]:
from urllib.parse import urlparse
from collections import namedtuple

def parse_sitename_from_url(url=None):
    """parse a Url into 4 components:
    <scheme>://<www.><node_name><extension><remove the rest>
    Return a 3-tuple: (www, node_name, extension)
    
    """
    #ParsedSitename = namedtuple('ParsedSitename', ['www', 'node_name','extension'])
    
    website_name = urlparse(url).netloc
    if website_name.startswith("www."):
        website_name = website_name[4:]
        www="www."
    node_name=website_name[:website_name.rfind('.')]
    #extension=website_name[website_name.rfind('.'):]
    
    return node_name
    #return ParsedSitename(www=www,node_name=node_name,extension=extension)

In [73]:
ReseauxList=["facebook","twitter", "youtube","plus.google","instagram","vimeo","linkedin","pinterest"]
BlackList=["vibes.yatego"]

In [75]:
Site_df = g.run(""" MATCH (a:Site) RETURN a.name""").to_data_frame()
Url_df = g.run(""" MATCH (a:Url) RETURN a.name""").to_data_frame()

In [68]:
write_db_node_from_url("https://www.liberation.fr/")

In [62]:
for url in Url_df["a.name"]:
    if parse_sitename_from_url(url) not in Site_df["a.name"].tolist():
        if parse_sitename_from_url(url) not in ReseauxList:
            if parse_sitename_from_url(url) not in BlackList:
                print(parse_sitename_from_url(url))

cse-cst.gc
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
itunes.apple
play.google
cargocollective
online-apothekekaufen
europecheapmeds
lariftcompagnie
vippharmacie
viagrabasprix
europe-med
viagrabasprix
acheterpilule
achetermedic
achatacheterprix
yoorshop
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
fr-contrainfo.espiv
newleftre

opa33
lignesdeforce.wordpress
demainlegrandsoir
archyves
lacanardesauvage.free
hors-sol
listes.rezo
lagazettedegouzy
regardeavue
canalsud
lalocale.ckdevelop
infoaut
antifa-net
parasite.antifa-net
lelotenaction
lille.indymedia
piecesetmaindoeuvre
cp-poligny.debatpublic
enigmail
lehollandaisvolant
gpg4win
jupiter1716.free
en.wikipedia
gpgtools
support.gpgtools
mailvelope
tails.boum
editions-albache
JefKlak
radiobartas
antifa-net
pidgin
otr.cypherpunks
atelier.mediaslibres
keepassx
guide.boum
info.securityinabox
guide.boum
tails.boum
mutu.mediaslibres
canalsud
mediaslibres
delairafranceinter.ouvaton
novofuck
lepost
mutu.mediaslibres
guide.boum
guide.boum
tails.boum
iaata
wiki.laquadrature
cnt-f
leprogres
cp-poligny.debatpublic
20minutes
legifrance.gouv
zyed-bouna-18-mai
contredit.blogspot
codedo.blogspot
actu.cotetoulouse
oclibertaire.free
fr-ca.facebook
tarnac.blog.lemonde
lignesdeforce.wordpress
pearltrees
sysiphus-angrynewsfromaroundtheworld.blogspot
culmine.noblogs
occupiedlondon
info

crashdebug
latribune
lesechos
francetvinfo
mac4ever
mac4ever
floss
bloomberg
lesjours
standard.co
lemediapresse
onpk
boingboing
carfree
cdn.radiofrance
cafepedagogique
revueperiode
images.vanityfair
vanityfair
lesechos
media.lesechos
africasacountry
stopauxviolences.blogspot
rtbf
ds1.static.rtbf
ds1.static.rtbf
ds1.static.rtbf
ds1.static.rtbf
ds1.static.rtbf
dedefensa
nonviolence21
freq-out
themesindep
ouvaton
ouvaton
ouvaton
grain
institut.inra
vacarme
listes.rezo
picoypala
lunivers
platenqmil
web.archive
revue-pouvoirs
cnil
sites.ina
rcf
april
cairn
kelli-arena
screenshots
who
whois.domaintools
en.wikipedia
shsu
en-marche
avenirencommun
marine2017
solutions-numeriques
google
ronan-chardonneau
service-public
lexpress
medium
radars-auto
provincedeliege
cavliege
francetvinfo
web.archive
april
april
privacytools
privatebrowsingmyths
ridt
aclu
aei
en.wikipedia
google
senat
futuropolis
feedburner.google
progresser-orthographe.lemonde
ws.amazon
shop.owni
fypeditions
petitcoucou.unblog
mediu

stage-photo
poaa.lenord
fr.wordpress
adobe
partageux.blogspot
accounts.google
accounts.google
accounts.google
google
accounts.google
accounts.google
google
accounts.google
accounts.google
accounts.google
softesthetique
accounts.google
accounts.google
accounts.google
accounts.google
accounts.google
accounts.google
horreursmusicales
accounts.google
accounts.google
accounts.google
accounts.google
accounts.google
accounts.google
toile-libre
gnu
4.bp.blogspot
regardeavue
1.bp.blogspot
1.bp.blogspot
droites-extremes.blog.lemonde
4.bp.blogspot
associationsalam
listes.rezo
antidollar.fr
antitravail.fr
issuu
whaller
arbreabulles
nationspresse
droites-extremes.blog.lemonde
i38.tinypic
convention-identitaire
blog.lefigaro
projet-apache
droites-extremes.blog.lemonde
4.bp.blogspot
1.bp.blogspot
bougnoulosophe.blogspot
fafbook
2.bp.blogspot
2.bp.blogspot
1.bp.blogspot
pvr-zone
fafbook
1.bp.blogspot
lepost
fdesouche
4.bp.blogspot
1.bp.blogspot
blog.slate
veilleur.blog.lemonde
paperblog
1.bp.blogspot


ricochets.des-blogueurs
alteretego.canalblog
les.grotesquent.free
sacripanne
samantdi
bricablog
blogskaia
valclair.canalblog
dotclear
yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
vibes.yatego
v

huffingtonpost
nouvelobs
telerama
sourcesure
mpublicite
snapchat
catalogue.immateriel
livre.immateriel
2018.capitoledulibre
developpez
bugs.debian
debian
zdnet
bits.debian
pix.toile-libre
wiki.debian
fluxbb
web.whatsapp
sites-internationaux
xn--changedeliens-9gb
themesaga
themesaga
joinmastodon
radical
radical
radical
radical
pixelfed
pixelfed
pixelfed
pixelfed
pixelfed
pixelfed
pixelfed
cafe.des-blogueurs
cafe.des-blogueurs
abos.mediapart
mastodon
mastodon
mastodon
mastodon
social.jesuislibre
social.jesuislibre
theconversation
japantimes.co
theconversation
birdsite
birdsite
birdsite
blog.spip
soc.psychedelic
soc.psychedelic
pbs.twimg
leravi
nytimes
static01.nyt
dailymail.co
i.dailymail.co
journals.openedition
openedition
nytimes
static01.nyt
dedefensa
francais.rt
positivr
positivr
cuisine-libre
francetvinfo
francetvinfo
embedftv-a.akamaihd
radioparleur
radioparleur
positifvortex.wordpress
cdn.radiofrance
taz
latribune
arretsurimages
medias.liberation
crashdebug
zettlr
zettlr
medium
co

google
google
google
google
google
google
google
google
google
google
google
google
google
google
google
img.over-blog-kiwi
img.over-blog-kiwi
img.over-blog-kiwi
communcommune
reveilcommuniste
canaille-le-rouge.over-blog
comite-valmy
actioncommuniste
pcfbassin
solidarite-internationale-pcf
prochetmoyen-orient
prochetmoyen-orient
prochetmoyen-orient
img.over-blog-kiwi
prochetmoyen-orient
prochetmoyen-orient
prochetmoyen-orient
prochetmoyen-orient
prochetmoyen-orient
prochetmoyen-orient
accounts.google
accounts.google
adelante
latinodocs.tumblr
republique-souveraine
frontsyndical-classe
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
img.over-blog-kiwi
fr.pinterest
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
jrcf.over-blog
jdgmedia
exalead
boardreader
lexpress
ticketac
itunes.apple
wikileaks
torproject
tails.boum
couragefound
wikileaks
wikileaks.spreadshirt
wikileaks
wikileaks
wikileaks
wikileaks
our.wikileaks
torproject
couragefo

In [118]:
write_db_node_from_url("https://www.liberation.fr/")

In [ ]:
for i in (g.run("MATCH (a:Url) RETURN a.name").data()):
    if parse_sitename_from_url(i['a.name']) not in Site_df["a.name"]:
        print(i)
        try:
            write_db_node_from_url(i['a.name'])
        except:
            continue

In [8]:
Site_df = g.run(""" MATCH (a:Site) RETURN a.name""").to_data_frame()
Url_df = g.run(""" MATCH (a:Url) RETURN a.name""").to_data_frame()


In [22]:
url="https://www.liberation.fr/"

write_db_node_from_url("https://www.liberation.fr/")

In [64]:
parse_sitename_from_url(url1)

'ww690.smartadserver'

In [8]:
for i in get_ext_links_df_from_url(url1)['href']:
    if (parse_sitename_from_url(url1)in parse_sitename_from_url(i))==False:
        print(i)

https://www.doctrine.fr/d/CA/Rennes/2019/C3F7559FF3D041362442C?q=CA%20Rennes%2027%20mars%202019%2016%2F03821&only_top_results=true&original_query_key=1f8c9974acfc6474b44ce0a10f8a55ce&selected_keywords=%5B%5D&position=1&query_key=1f8c9974acfc6474b44ce0a10f8a55ce&source=excerpt_results
https://www.legifrance.gouv.fr/affichCodeArticle.do?cidTexte=LEGITEXT000006070721&idArticle=LEGIARTI000006435367
https://www.barnebys.fr/ventes/bijouterie-et-joaillerie/
https://www.barnebys.fr/ventes/bijouterie-et-joaillerie/
https://www.courrierinternational.com/
http://www.lavie.fr/
https://www.huffingtonpost.fr/
https://www.nouvelobs.com/
https://www.monde-diplomatique.fr/
https://www.telerama.fr/
https://www.sourcesure.eu/
http://mpublicite.fr/
https://www.facebook.com/lemonde.fr
https://www.youtube.com/user/LeMonde
https://twitter.com/lemondefr
https://www.instagram.com/lemondefr/?hl=fr
https://www.snapchat.com/discover/Le-Monde/8843708388


In [71]:
parse_sitename_from_url("https://twitter.com/joweisz")

'twitter'